## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_to_load = "../Weather_Database/WeatherPy_Database.csv"
city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nanortalik,GL,60.1432,-45.2371,39.51,98,100,6.96,overcast clouds
1,1,Yellowknife,CA,62.4560,-114.3525,73.92,20,20,11.50,few clouds
2,2,East London,ZA,-33.0153,27.9116,61.95,61,100,6.33,overcast clouds
3,3,Pakxan,LA,18.3778,103.6601,80.85,82,84,2.55,broken clouds
4,4,Nadapuram,IN,11.7000,75.6667,75.76,90,100,1.72,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Yellowknife,CA,62.4560,-114.3525,73.92,20,20,11.50,few clouds
3,3,Pakxan,LA,18.3778,103.6601,80.85,82,84,2.55,broken clouds
4,4,Nadapuram,IN,11.7000,75.6667,75.76,90,100,1.72,light rain
5,5,Bethel,US,41.3712,-73.4140,66.29,38,40,4.61,scattered clouds
8,8,Mahebourg,MU,-20.4081,57.7000,66.25,72,10,10.13,clear sky
10,10,Bambous Virieux,MU,-20.3428,57.7575,66.22,73,8,12.30,clear sky
11,11,Kapaa,US,22.0752,-159.3190,84.18,79,75,13.80,broken clouds
13,13,Pemangkat,ID,1.1667,108.9667,79.70,82,7,4.50,clear sky
19,19,Vaini,TO,-21.2000,-175.2000,82.56,83,40,14.97,scattered clouds
20,20,Rikitea,PF,-23.1203,-134.9692,72.07,66,91,22.82,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Yellowknife,CA,73.92,few clouds,62.4560,-114.3525,
3,Pakxan,LA,80.85,broken clouds,18.3778,103.6601,
4,Nadapuram,IN,75.76,light rain,11.7000,75.6667,
5,Bethel,US,66.29,scattered clouds,41.3712,-73.4140,
8,Mahebourg,MU,66.25,clear sky,-20.4081,57.7000,
10,Bambous Virieux,MU,66.22,clear sky,-20.3428,57.7575,
11,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,
13,Pemangkat,ID,79.70,clear sky,1.1667,108.9667,
19,Vaini,TO,82.56,scattered clouds,-21.2000,-175.2000,
20,Rikitea,PF,72.07,overcast clouds,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Yellowknife,CA,73.92,few clouds,62.4560,-114.3525,The Explorer Hotel
3,Pakxan,LA,80.85,broken clouds,18.3778,103.6601,Kong Kham Pheng Guesthouse
4,Nadapuram,IN,75.76,light rain,11.7000,75.6667,Fabina Garden
5,Bethel,US,66.29,scattered clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
8,Mahebourg,MU,66.25,clear sky,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))